In [1]:
# Import Dependencies
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, text
import geopandas as gpd 

# Generic and custom queries to the database

In [2]:
# Connect to database
database_path = Path("data.sqlite")

engine = create_engine(f"sqlite:///{database_path}")

In [3]:
# Get GeoJSON data for state and county boundaries

states = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_state_500k.zip")
counties = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_500k.zip")

In [4]:
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,20,161,00485044,0500000US20161,20161,Riley,Riley County,KS,Kansas,06,1579077672,32047392,"POLYGON ((-96.96095 39.28670, -96.96106 39.288..."
1,19,159,00465268,0500000US19159,19159,Ringgold,Ringgold County,IA,Iowa,06,1386932347,8723135,"POLYGON ((-94.47167 40.81255, -94.47166 40.819..."
2,30,009,01720111,0500000US30009,30009,Carbon,Carbon County,MT,Montana,06,5303728455,35213028,"POLYGON ((-109.79867 45.16734, -109.68779 45.1..."
3,16,007,00395090,0500000US16007,16007,Bear Lake,Bear Lake County,ID,Idaho,06,2527123155,191364281,"POLYGON ((-111.63452 42.57034, -111.63010 42.5..."
4,55,011,01581065,0500000US55011,55011,Buffalo,Buffalo County,WI,Wisconsin,06,1750290818,87549529,"POLYGON ((-92.08384 44.41200, -92.08310 44.414..."


In [5]:
# Keep only main states data
states = states[(states["GEOID"] <= "56")]

counties = counties[(counties["STATEFP"] <= "56")]

## Generic queries

In [6]:
# Get list of states
def get_states(): 
    select_statement = f"""
    SELECT state_code,
           state_name
        FROM state
    """
    print(select_statement) 

    with engine.connect() as connection:
         
        query = text(select_statement)
        result = connection.execute(query)

        result_list = [dict(row) for row in result]
    return result_list



In [7]:
get_states()


    SELECT state_code,
           state_name
        FROM state
    


[{'state_code': 'US', 'state_name': 'All states'},
 {'state_code': 'AL', 'state_name': 'Alabama'},
 {'state_code': 'AK', 'state_name': 'Alaska'},
 {'state_code': 'AZ', 'state_name': 'Arizona'},
 {'state_code': 'AR', 'state_name': 'Arkansas'},
 {'state_code': 'CA', 'state_name': 'California'},
 {'state_code': 'CO', 'state_name': 'Colorado'},
 {'state_code': 'CT', 'state_name': 'Connecticut'},
 {'state_code': 'DE', 'state_name': 'Delaware'},
 {'state_code': 'DC', 'state_name': 'District of Columbia'},
 {'state_code': 'FL', 'state_name': 'Florida'},
 {'state_code': 'GA', 'state_name': 'Georgia'},
 {'state_code': 'HI', 'state_name': 'Hawaii'},
 {'state_code': 'ID', 'state_name': 'Idaho'},
 {'state_code': 'IL', 'state_name': 'Illinois'},
 {'state_code': 'IN', 'state_name': 'Indiana'},
 {'state_code': 'IA', 'state_name': 'Iowa'},
 {'state_code': 'KS', 'state_name': 'Kansas'},
 {'state_code': 'KY', 'state_name': 'Kentucky'},
 {'state_code': 'LA', 'state_name': 'Louisiana'},
 {'state_code': 'M

In [9]:
def get_industries():
    select_statement = f"""
    SELECT industry_code,
           industry_name
        FROM industry
    """
    print(select_statement) 

    with engine.connect() as connection:
         
        query = text(select_statement)
        result = connection.execute(query)

        result_list = [dict(row) for row in result]
    return result_list


In [12]:
get_industries()





    SELECT industry_code,
           industry_name
        FROM industry
    


[{'industry_code': 1011,
  'industry_name': '1011 - National resources and mining'},
 {'industry_code': 1012, 'industry_name': '1012 - Construction'},
 {'industry_code': 1013, 'industry_name': '1013 - Manufacturing'},
 {'industry_code': 1021,
  'industry_name': '1021 - Trade, transportation, and utilities'},
 {'industry_code': 1022, 'industry_name': '1022 - Information'},
 {'industry_code': 1023, 'industry_name': '1023 - Financial activities'},
 {'industry_code': 1024,
  'industry_name': '1024 - Professional and business services'},
 {'industry_code': 1026, 'industry_name': '1026 - Leisure and hospitality'},
 {'industry_code': 1027, 'industry_name': '1027 - Other services'},
 {'industry_code': 1028, 'industry_name': '1028 - Public administration'},
 {'industry_code': 1029, 'industry_name': '1029 - Unclassified'}]

In [17]:
reduction = 5
def get_employment_map(state_code, industry_code, reduction):
   select_statement = ""

   if state_code == "US":
      select_statement = f"""
        WITH 
        industry_employment 
        AS
        (
            SELECT c.state_code, 
                    SUM(bls_annual_employment) AS industry_employment
                FROM county_industry_metric cim
                        INNER JOIN
                        county c ON c.county_fips = cim.county_fips
                WHERE industry_code = {industry_code}
                    AND year = (SELECT MAX(year) FROM county_industry_metric)
            GROUP BY c.state_code
        ),
        state_employment 
        AS
        (
            SELECT c.state_code, 
                    SUM(bls_annual_employment) AS total_employment
                FROM county_industry_metric cim
                        INNER JOIN
                        county c ON c.county_fips = cim.county_fips
                WHERE year = (SELECT MAX(year) FROM county_industry_metric)
            GROUP BY c.state_code
        )
        SELECT se.state_code,
                se.total_employment,
                ie.industry_employment
            FROM state_employment se
                    INNER JOIN
                    industry_employment ie ON ie.state_code = se.state_code
        """
   else:
        select_statement = f"""
        WITH 
        industry_employment
        AS
        (
            SELECT c.county_name,
                    c.state_code,
                    bls_annual_employment AS industry_employment
                FROM county_industry_metric cim
                        INNER JOIN
                        county c ON c.county_fips = cim.county_fips
                WHERE industry_code = {industry_code}
                    AND year = (SELECT MAX(year) FROM county_industry_metric)
                    AND state_code = '{state_code}'
        ),
        county_employment
        AS
        (
            SELECT c.county_name,
                    c.state_code,
                    SUM(bls_annual_employment) AS total_employment
                FROM county_industry_metric cim
                        INNER JOIN
                        county c ON c.county_fips = cim.county_fips
                WHERE year = (SELECT MAX(year) FROM county_industry_metric)
                    AND state_code = '{state_code}'
            GROUP BY c.county_name, c.state_code
        )
        SELECT ce.county_name,
                ce.state_code,
                ce.total_employment,
                ie.industry_employment
            FROM county_employment ce
                    INNER JOIN
                    industry_employment ie ON ie.county_name = ce.county_name
        """
print(select_statement)

# Exexute the query
with engine.connect() as connection:
     query = text(select_statement)
     result = connection.execute(query)

     result_list = [dict(row) for row in result]

# Convert result to DataFrame 
df = pd.DataFrame(result_list)

# Calculate current share of industry by employment
df["current_industry_share"] = 100 * df["industry_employment"] / df["total_employment"]

# Calculate reduced share of industry by employment
reduction_rate = (100 - reduction) / 100
df["reduction_industry_share"] = 100 * (reduction_rate * df["industry_employment"]) / (df["total_employment"] - ((reduction / 100) * df["industry_employment"]))

# Adding geo data
# Depending of the area selection (national or particular state), creating geoPandas DataFrame

if state_code == "US":
   # Create a copy of states GeoPandas DataFrame
   geoPandas = states.copy

   # Merge states geoJSON with a metric
   geoPandas = geoPandas.merge(df, how="inner", left_on="STUSPS", right_on = "state_code")

   # Drop unnecessary column
   geoPandas.drop(columns="state_code", inplace=True)
else:
   # Create a copy of states GeoPandas DataFrame
   geoPandas = states.copy

   # Merge states geoJSON with a metric
   geoPandas = geoPandas.merge(df, how="inner", left_on=["STUSPS", "NAME"], right_on = ["state_code", "county_name"])

   # Drop unnecessary column
   geoPandas.drop(columns=["state_code", "county_name"], inplace=True)

   # Convert results to GeoJSON
return geoPandas.to_json()

           


NameError: name 'select_statement' is not defined

In [20]:
get_employment_map(state_code="CA", industry_code=1011, reduction=15)


        WITH 
        industry_employment
        AS
        (
            SELECT c.county_name,
                    c.state_code,
                    bls_annual_employment AS industry_employment
                FROM county_industry_metric cim
                        INNER JOIN
                        county c ON c.county_fips = cim.county_fips
                WHERE industry_code = 1011
                    AND year = (SELECT MAX(year) FROM county_industry_metric)
                    AND state_code = 'CA'
        ),
        county_employment
        AS
        (
            SELECT c.county_name,
                    c.state_code,
                    SUM(bls_annual_employment) AS total_employment
                FROM county_industry_metric cim
                        INNER JOIN
                        county c ON c.county_fips = cim.county_fips
                WHERE year = (SELECT MAX(year) FROM county_industry_metric)
                    AND state_code = 'CA'
            GROUP BY c.coun

In [21]:
def get_employment_trend(state_code, industry_code, reduction):

    if state_code == "US":
        select_statement = f"""
        SELECT year,
               SUM(bls_annual_employment) AS metric
            FROM county_industry_metric
            WHERE industry_code = {industry_code}
        GROUP BY year
        """
    else:
        select_statement = f"""
        SELECT m.year,
               SUM(bls_annual_employment) AS metric
            FROM county_industry_metric m
                INNER JOIN
                county c ON c.county_fips = m.county_fips
            WHERE c.state_code = '{state_code}'
              AND industry_code = {industry_code}
        GROUP BY m.year
        """
    print(select_statement)
    with engine.connect() as connection:
        query = text(select_statement)
        result = connection.execute(query)

        result_list = [dict(row) for row in result]
    print(result_list)

    latest_employment = result_list[-1]["metric"]

    # Calculate forecasted employment based on the latest employment and reduction
    employment2030 = latest_employment * (100 - reduction) / 100

    # Append the forecasted employment to the results
    result_list.append(dict({'year': 2030, 'metric': employment2030}))
    return result_list
    
   

In [22]:
get_employment_trend(state_code="CA", industry_code=1011, reduction=15)


        SELECT m.year,
               SUM(bls_annual_employment) AS metric
            FROM county_industry_metric m
                INNER JOIN
                county c ON c.county_fips = m.county_fips
            WHERE c.state_code = 'CA'
              AND industry_code = 1011
        GROUP BY m.year
        
[{'year': 2020, 'metric': 423704}, {'year': 2021, 'metric': 426363}, {'year': 2022, 'metric': 434918}]


[{'year': 2020, 'metric': 423704},
 {'year': 2021, 'metric': 426363},
 {'year': 2022, 'metric': 434918},
 {'year': 2030, 'metric': 369680.3}]

In [27]:
def get_unemployment_rate(state_code, industry_code, reduction): 
    # Check the area level
    if state_code == "US":
        select_statement = f"""
        WITH 
        industry_employment
        AS
        (
            SELECT s.state_name,
                   SUM(bls_annual_employment) AS industry_employment
                FROM county_industry_metric cim
                     INNER JOIN
                     county c ON c.county_fips = cim.county_fips
                     INNER JOIN
                     state s ON s.state_code = c.state_code
                WHERE industry_code = {industry_code}
                  AND year = (SELECT MAX(year) FROM county_industry_metric)
            GROUP BY s.state_name
        ),
        state_employment
        AS
        (
            SELECT s.state_name,
                   SUM(bls_labor_force) AS labor_force,
                   SUM(bls_employed) AS employment
                FROM county_metric m
                     INNER JOIN
                     county c ON c.county_fips = m.county_fips
                     INNER JOIN
                     state s ON s.state_code = c.state_code
                WHERE m.year = (SELECT MAX(year) FROM county_metric)
            GROUP BY s.state_name
        )
        SELECT se.state_name AS area_name,
               se.labor_force,
               se.employment,
               ie.industry_employment
            FROM state_employment se
                 INNER JOIN
                 industry_employment ie ON ie.state_name = se.state_name
        """
    else:
        select_statement = f"""
        WITH 
        industry_employment
        AS
        (
            SELECT c.county_name,
                   bls_annual_employment AS industry_employment
                FROM county_industry_metric cim
                     INNER JOIN
                     county c ON c.county_fips = cim.county_fips
                WHERE industry_code = {industry_code}
                  AND year = (SELECT MAX(year) FROM county_metric)
                  AND state_code = '{state_code}'
        ),
        county_employment
        AS
        (
            SELECT c.county_name,
                   bls_labor_force AS labor_force,
                   bls_employed AS employment
                FROM county_metric m
                     INNER JOIN
                     county c ON c.county_fips = m.county_fips
                WHERE m.year = (SELECT MAX(year) FROM county_metric)
                  AND state_code = '{state_code}'
        )
        SELECT ce.county_name AS area_name,
               ce.labor_force,
               ce.employment,
               ie.industry_employment
            FROM county_employment ce
                 INNER JOIN
                 industry_employment ie ON ie.county_name = ce.county_name
        """
print(select_statement)

with engine.connect() as connection:
        query = text(select_statement)
        result = connection.execute(query)

        result_list = [dict(row) for row in result]

df = pd.DataFrame(result_list)

# Calculate current employment rate
df["unemployment_rate"] = 100 * (df["labor_force"] / df["employment"]) / df["labor_force"]

# Calculate average current employment rate

df["average_unemployment_rate"] = 100 * (df["labor_force"].sum() / df["employment"].sum()) / df["labor_force"].sum()

# Calculate forecast unemployment rate, if employment in a particular indusrtry will be reduced

df["forecasted_unemployment_rate"] = 100 * (df["labor_force"] - (df["employment"] - (reduction / 100) * df["industry_employment"])) / df["labor_force"] 
return df.to_dict('records')  

NameError: name 'select_statement' is not defined

In [28]:
get_unemployment_rate(state_code="CA", industry_code=1011, reduction=15)

In [30]:
if state_code == "US":
        select_statement = f"""
        -- Мы должны посчитать общий доход людей на уровне штата или каунти:
        -- Важно понимать, что хотим вытащить:
        WITH 
        industry_income
        AS
        (
            SELECT c.state_code,
                   SUM(bls_total_annual_wages) AS industry_wage, --и сколько они всего заработали зарплат
                   SUM(bls_annual_employment) AS industry_employment -- должны посчитать, сколько людей работало в конкретной отрасли в году в этом штате  
                FROM county_industry_metric cim
                     INNER JOIN
                     county c ON c.county_fips = cim.county_fips
                WHERE industry_code = {industry_code} -- фильтр означает, что вытаскиваем данные для конкретной индустрии
                  AND year = (SELECT MAX(year) FROM county_industry_metric) -- вытащить данные за последний год из таблички. Можно написать year = 2022
            GROUP BY c.state_code
        ),
        state_income -- сколько всего людей работали в конкретной отрасли, сколько они заработали зарплат 
        -- получили: в этом штате в этом году жило 1 млн человек, они в сумме заработали миллиард
       
        AS
        (   -- второй запрос - считаем по всем штатам
            SELECT c.state_code, -- теперь считаем по всем штатам сумму общего дохода по каждому штату
                   SUM(bea_total_income) AS total_income, --поскольку данные представлены на уровне каунти, то мы должны просуммировать. 
                   SUM(population) AS population -- Нужен population (сколько народа в этом штате жило в конкретный год)
                FROM county_metric m
                     INNER JOIN
                     county c ON c.county_fips = m.county_fips
                WHERE m.year = (SELECT MAX(year) FROM county_metric)
            GROUP BY c.state_code
        )
        -- после чего два CTE джойним и вытаскиваем код штата, общий доход всех жителей, количество населения в этом штате, 
        --суммарная зарплата всех работников в штате в этой отрасли industry_wage, и сколько всего в этой отрасли работало людей в этом году industry_employment
        
        SELECT si.state_code,
               si.total_income,
               si.population,
               ii.industry_wage,
               ii.industry_employment
            FROM state_income si
                 INNER JOIN
                 industry_income ii ON ii.state_code = si.state_code
        """
    else:
        select_statement = f"""
        WITH 
        industry_income
        AS
        (
            SELECT c.county_name,
                   c.state_code,
                   bls_total_annual_wages AS industry_wage,
                   bls_annual_employment AS industry_employment
                FROM county_industry_metric cim
                     INNER JOIN
                     county c ON c.county_fips = cim.county_fips
                WHERE industry_code = {industry_code}
                  AND year = (SELECT MAX(year) FROM county_metric)
                  AND state_code = '{state_code}'
        ),
        county_income
        AS
        (
            SELECT c.county_name,
                   c.state_code,
                   bea_total_income AS total_income,
                   population
                FROM county_metric m
                     INNER JOIN
                     county c ON c.county_fips = m.county_fips
                WHERE m.year = (SELECT MAX(year) FROM county_metric)
                  AND state_code = '{state_code}'
        )
        SELECT ci.county_name,
               ci.state_code,
               ci.total_income,
               ci.population,
               ii.industry_wage,
               ii.industry_employment
            FROM county_income ci
                 INNER JOIN
                 industry_income ii ON ii.county_name = ci.county_name
        """


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 45)